In [7]:
!pip install jugaad_data yfinance

  Using cached jugaad_data-0.23-py3-none-any.whl (26 kB)
  Created wheel for get-all-tickers: filename=get_all_tickers-1.8-py3-none-any.whl size=4247 sha256=2d9509f94da2b2b076685ca999063bcdbedc2d448cc636e88a181362cfd36299
  Stored in directory: /home/jovyan/.cache/pip/wheels/11/17/a3/80479161a3294cdda8205fdb6bb28a45298c0a4248d5cfcb2a
Successfully built get-all-tickers
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/opt/conda/lib/python3.8/site-packages/tests/__init__.py'
Consider using the `--user` option or check the permissions.



In [10]:
from datetime import date
from jugaad_data import nse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import csv
import yfinance

#### Get stock tickers

In [3]:
# Futures
'''
# Gather all symbols from CSV
with open('fo_mktlots.csv') as fp:
    reader = csv.reader(fp)
    rows = list(reader)
    # stocks start from 5th row and symbols are in 2nd column
    fno_stocks = [row[1].lstrip().rstrip() for row in rows[5:]]
    
print("Stocks in F&O: {}".format(len(fno_stocks)))
'''

Stocks in F&O: 188


In [41]:
# Stocks from NYSE

# Gather all symbols from CSV
stocks = pd.DataFrame(columns=['Ticker', 'Close', 'Market Cap'])
with open('stock_symbols.csv') as f:
    reader = csv.reader(f)
    rows = list(reader)
    
    for i, row in enumerate(rows):
        if i == 0 or row[2] == 'n/a':
            continue
        try:
            row = pd.Series([row[0].lstrip().rstrip(), float(row[2]), row[3]], index=stocks.columns)
            stocks = stocks.append(row, ignore_index=True)
        except:
            print('Error with ticker:', row[0])
    
print("Stocks in our NYSE: {}".format(len(stocks)))

Error with ticker: NRKM
Error with ticker: GFNCZ
Error with ticker: LUCA
Error with ticker: RPRXW
Error with ticker: RPRXZ
Error with ticker: ROIQ
Error with ticker: ROIQW
Error with ticker: STD^f
Error with ticker: CBO
Error with ticker: ZY^A
Error with ticker: AAZ^K
Error with ticker: ZYZ^K
Error with ticker: DDZ^K
Error with ticker: FFZ^K
Error with ticker: IIZ^K
Error with ticker: ZYY
Error with ticker: ZYY^A
Error with ticker: ZZA
Error with ticker: ZZB
Error with ticker: ZZD
Error with ticker: ZZE
Error with ticker: ZZF
Error with ticker: ZZG
Error with ticker: ZZH
Error with ticker: ZZI
Error with ticker: ZZJ
Error with ticker: IBO
Error with ticker: WWZ^K
Error with ticker: MMZ^K
Error with ticker: OOZ^K
Error with ticker: QQZ^K
Error with ticker: SS^K
Error with ticker: ZXX
Error with ticker: ZXX^F
Error with ticker: TTTM/WS/W
Error with ticker: TTTM/WS/Z
Stocks in our NYSE: 6515


In [42]:
stocks.head()

,Ticker,Close,Market Cap
0,AAPL,565.25,"$527,022,142,500"
1,XOM,84.57,"$397,687,211,340"
2,MSFT,30.98,"$259,946,085,580"
3,BAC^I,24.79,"$248,692,932,940"
4,IBM,204.99,"$236,452,685,160"


In [45]:
# Get Market Cap as a float instead of a currency string
stocks['Market Cap'] = stocks['Market Cap'].replace('\$|,', '', regex=True).astype(float)

#### Filter stocks to only show certain (at least somewhat reliable companies)

In [47]:
MIN_MARKET_CAP = 1000000
MIN_CLOSE = 50
MIN_AVG_VOLUME = 100000

stocks = stocks[(stocks['Market Cap'] > MIN_MARKET_CAP) & (stocks['Close'] > MIN_CLOSE)]

In [48]:
print("Down to {} stocks in our NYSE".format(len(stocks)))

Down to 658 stocks in our NYSE


In [59]:
stocks.head()

,Ticker,Close,Market Cap
0,AAPL,565.25,5.270221e+11
1,XOM,84.57,3.976872e+11
4,IBM,204.99,2.364527e+11
5,CVX,103.72,2.049561e+11
7,WMT,58.70,1.996196e+11


#### Download stock data for each ticker

In [78]:
def get_stock_data(stonks, from_date=date(2019,1,1), to_date=date(2021,3,1)):
    dfs = []

    for ticker in tqdm(stonks['Ticker']):
        try:
            ticker = ticker.replace("/", "-").replace("^", "-")
            df = yfinance.Ticker(ticker).history(start=from_date, end=to_date)
            if not df or len(df) == 0:
                continue
            dfs.append(df)
        except:
            print("Issue retrieving stock data for {}".format(ticker))

    print()
    print(dfs[0].head())
    return dfs

In [79]:
dfs = get_stock_data(stocks)

  0%|          | 0/658 [00:00<?, ?it/s]

- UTX: No data found, symbol may be delisted
- ESRX: No data found for this date range, symbol may be delisted
- MON: Data doesn't exist for startDate = 1546300800, endDate = 1614556800
- PCLN: No data found for this date range, symbol may be delisted
- APC: No data found, symbol may be delisted
- PX: Data doesn't exist for startDate = 1546300800, endDate = 1614556800
- CELG: No data found, symbol may be delisted
- AGN: No data found, symbol may be delisted
- TYC: No data found for this date range, symbol may be delisted
- TWC: No data found for this date range, symbol may be delisted
- WLP: No data found for this date range, symbol may be delisted
- COH: No data found for this date range, symbol may be delisted
- KMP: No data found for this date range, symbol may be delisted
- SHPGY: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- WPZ: No data found for this date range, symbol may be delisted
- LO: No data found for this date ra

In [86]:
dfs[0].head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,37.590927,38.551998,37.430747,38.326290,148158800,0.0,0.0
2019-01-03,34.943126,35.365416,34.462593,34.508705,365248800,0.0,0.0
2019-01-04,35.076612,36.052245,34.899446,35.981861,234428400,0.0,0.0
2019-01-07,36.088647,36.120199,35.409102,35.901772,219111200,0.0,0.0
2019-01-08,36.297371,36.845863,36.044970,36.586178,164101200,0.0,0.0


Again, filter down our data. This time, we only want to look at stocks that have a "decent" volume.

In [11]:
MIN_AVG_VOLUME = 100000
def valid_stock(stock_data):
    return True if stock_data["Volume"].mean() > MIN_AVG_VOLUME else False


In [95]:
# Combine data into one giant dataframe
all_stock_df = pd.DataFrame()
for i, df in enumerate(dfs):
    # Since we couldn't get data for many companies, fill in those removed rows by updating line numbers
    stocks = stocks.reset_index(drop=True)
    if valid_stock(stocks):
        all_stock_df[stocks["Ticker"][i]] = df['Close']
    
print(all_stock_df.head())

                 AAPL        XOM        IBM        CVX        WMT         JNJ  \
Date                                                                            
2019-01-02  38.326290  58.018681  95.081238  95.651024  88.933044  117.985733   
2019-01-03  34.508705  57.127884  93.183083  93.819054  88.475716  116.110893   
2019-01-04  35.981861  59.234177  96.822594  95.763359  89.028328  118.059624   
2019-01-07  35.901772  59.542210  97.507576  97.007721  90.076401  117.302284   
2019-01-08  36.586178  59.975132  98.894058  96.584297  90.705238  120.026825   

                   PG         KO         GOOG         PM  ...        SYT  \
Date                                                      ...              
2019-01-02  84.517372  42.639236  1045.849976  57.416683  ...  71.100281   
2019-01-03  83.924782  42.375751  1016.059998  56.674561  ...  71.904358   
2019-01-04  85.637711  43.220726  1070.709961  59.327461  ...  71.062431   
2019-01-07  85.295143  42.657413  1068.390015  57.51

In [96]:
# Save our list of stock data
all_stock_df.to_csv("nyse_raw_data.csv", index=False)

In [14]:
all_stock_df = pd.read_csv("nyse_raw_data.csv")

In [15]:
stock_corr = all_stock_df.corr()
print(stock_corr.head())

          AAPL       XOM       IBM       CVX       WMT       JNJ        PG  \
AAPL  1.000000 -0.746746 -0.166341 -0.651332  0.936966  0.866409  0.885792   
XOM  -0.746746  1.000000  0.576170  0.962617 -0.774981 -0.548785 -0.680790   
IBM  -0.166341  0.576170  1.000000  0.683296 -0.173503 -0.003964  0.016960   
CVX  -0.651332  0.962617  0.683296  1.000000 -0.650419 -0.452349 -0.538675   
WMT   0.936966 -0.774981 -0.173503 -0.650419  1.000000  0.778405  0.934121   

            KO      GOOG        PM  ...       SYT      LQDT      CSTR  \
AAPL  0.284339  0.936794  0.440295  ...  0.727292  0.983568  0.810380   
XOM   0.032962 -0.563692  0.069472  ... -0.919691 -0.644375 -0.270910   
IBM   0.566968 -0.031623  0.504911  ... -0.382278 -0.048394  0.158598   
CVX   0.196846 -0.455807  0.149181  ... -0.818006 -0.534591 -0.180453   
WMT   0.353648  0.858633  0.334302  ...  0.790548  0.920289  0.708716   

           BBL       CKH      ANAT       CRR       DSW        EC      MIDD  
AAPL  0.828937 

#### Filter to only show very high/low correlation

In [99]:
pos_threshold, neg_threshold = 0.95, 0.8
for stock in all_stock_df:
    k = stock_corr[(stock_corr[stock] > pos_threshold) | (stock_corr[stock] < neg_threshold)][stock]
    if k.any():
        print(k.to_frame().T)
        print('------------------------------------\n\n')

           ACN      GILD       APC         D        GD       PGN      CHKP  \
AAPL  0.973543  0.955725  0.989881  0.955998  0.977455  0.967587  0.965114   

           ZMH       RRC       BCR  ...       OIS       WBC       KEX  \
AAPL  0.955349  0.958568  0.963232  ...  0.962039  0.958415  0.956786   

          SIVB     PSMT       DTG      CVLT     MLNX      WXS      LQDT  
AAPL  0.951619  0.96077  0.954874  0.961558  0.95597  0.97593  0.983568  

[1 rows x 23 columns]
------------------------------------


          CVX       SLB       UTX       AXP        CL      BIIB      CELG  \
XOM  0.962617  0.966296  0.952148  0.954088  0.958499  0.980392  0.962315   

           CM       CCI        GR     CTXS       CLB       MSM  
XOM  0.958092  0.978732  0.969926  0.96291  0.968364  0.950748  
------------------------------------


          XOM       SPG
CVX  0.962617  0.950386
------------------------------------


          CNI       BCR
WMT  0.976676  0.953798
---------------------------

KeyError: 'UHAL'

#### Let's explore the industry/sector of these highly correlated stocks

In [18]:
# for stock in all_stock_df:
#     t = yfinance.Ticker(stock)
#     print("Ticker:", stock, "Industry:", t.info["sector"])
#     break

Industry: Technology


#### Collect stats on our pairs to further investigate possible pairs

In [ ]:
# First, we need to have a dictionary/2d array which can list all possible pairs
# Then, we can calculate the ratios to all these stocks for each day
ratios = stock_1['CLOSE'] / stock_2['CLOSE']

Finally, with these statistics calculated, the last bit of information we want to add to our feature space will be social media sentiment. Will these correlated stocks have early warning indicators from social media? That's what we aim to find out!